# Data Workflow: NYC Airbnb Open Data Analysis

**Author:** Tyler Wilcoxson
**Dataset:** NYC Airbnb Open Data (2019) — 48,895 listings across five boroughs
**Date:** February 2026

This project demonstrates a reproducible data workflow using Python, Pandas, and
Seaborn. We ingest, clean, explore, and visualize the NYC Airbnb Open Data to
uncover pricing patterns, geographic distributions, and room-type compositions
across New York City's five boroughs.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import seaborn as sns

# Display options
pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 120)
pd.set_option("display.float_format", "{:.2f}".format)

# Plot style
sns.set_theme(style="whitegrid", font_scale=1.1)
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["figure.dpi"] = 100

print("Setup complete.")

## 1. Data Ingestion

We load the CSV directly from the project directory. The dataset was originally
published on Kaggle by Dgomonov and contains information about Airbnb listings
in New York City for 2019.

In [ ]:
df_raw = pd.read_csv("AB_NYC_2019.csv")

print(f"Shape: {df_raw.shape[0]:,} rows × {df_raw.shape[1]} columns\n")
df_raw.head(10)

In [ ]:
print("Data types:")
print(df_raw.dtypes)
print(f"\nMemory usage: {df_raw.memory_usage(deep=True).sum() / 1e6:.1f} MB")

In [ ]:
print("Missing values per column:\n")
missing = df_raw.isnull().sum()
print(missing[missing > 0].sort_values(ascending=False))
print(f"\nTotal missing cells: {df_raw.isnull().sum().sum():,}")

In [ ]:
df_raw.describe()

### Observations on Raw Data

- **48,895 rows** and **16 columns** loaded successfully.
- `reviews_per_month` has ~10,052 nulls — these correspond to listings with zero
  reviews that have never been reviewed.
- `last_review` has the same ~10,052 nulls, consistent with the above.
- `name` (16 nulls) and `host_name` (21 nulls) have minor missing values.
- The `price` column ranges from \$0 to \$10,000, suggesting outliers at both ends.
- `minimum_nights` reaches 1,250 days, indicating some listings are effectively
  long-term rentals rather than typical short-stay Airbnb units.

## 2. Data Cleaning

Data cleaning follows tidy data principles (Wickham, 2014), ensuring each
variable forms a column, each observation forms a row, and each type of
observational unit forms a table. We address missing values, parse date
columns, and remove price outliers that would distort analysis.

In [ ]:
def handle_missing_values(df):
    """Handle missing values in the Airbnb dataset.

    Strategy:
    - reviews_per_month: Fill with 0.0 (no reviews means zero monthly rate).
    - last_review: Parse to datetime; NaT for never-reviewed listings is valid.
    - name / host_name: Fill with 'Unknown' to preserve row count.

    Parameters
    ----------
    df : pd.DataFrame
        Raw Airbnb dataframe.

    Returns
    -------
    pd.DataFrame
        Dataframe with missing values handled.
    """
    df = df.copy()
    df["reviews_per_month"] = df["reviews_per_month"].fillna(0.0)
    df["last_review"] = pd.to_datetime(df["last_review"], format="mixed")
    df["name"] = df["name"].fillna("Unknown")
    df["host_name"] = df["host_name"].fillna("Unknown")
    return df

In [ ]:
def remove_price_outliers(df, lower=1, upper_quantile=0.99):
    """Remove price outliers from the dataset.

    Listings priced at $0 are likely errors or inactive entries. Extreme high
    prices above the 99th percentile are removed to prevent skewing
    summary statistics and visualizations.

    Parameters
    ----------
    df : pd.DataFrame
        Airbnb dataframe with a 'price' column.
    lower : int or float
        Minimum valid price (exclusive lower bound for removal). Default 1.
    upper_quantile : float
        Quantile threshold for upper bound. Default 0.99.

    Returns
    -------
    pd.DataFrame
        Filtered dataframe with outliers removed.
    """
    upper_bound = df["price"].quantile(upper_quantile)
    mask = (df["price"] >= lower) & (df["price"] <= upper_bound)
    removed = len(df) - mask.sum()
    print(f"Upper bound (p{upper_quantile*100:.0f}): ${upper_bound:,.0f}")
    print(f"Rows removed: {removed:,} ({removed/len(df)*100:.1f}%)")
    return df[mask].reset_index(drop=True)

In [ ]:
print(f"Before cleaning: {df_raw.shape[0]:,} rows")
print(f"Missing values: {df_raw.isnull().sum().sum():,}\n")

df = handle_missing_values(df_raw)
print(f"After handling missing values: {df.isnull().sum().sum()} missing cells")
print(f"last_review dtype: {df['last_review'].dtype}\n")

df = remove_price_outliers(df)
print(f"\nAfter cleaning: {df.shape[0]:,} rows")

### Cleaning Summary

| Issue | Decision | Rationale |
|---|---|---|
| `reviews_per_month` nulls | Filled with 0.0 | Nulls represent listings with no reviews; zero is semantically correct |
| `last_review` nulls | Kept as NaT after datetime parsing | Absence of a review date is meaningful, not an error |
| `name` / `host_name` nulls | Filled with "Unknown" | Preserves row count for numeric analysis |
| Price = $0 | Removed | Likely inactive or erroneous listings |
| Price > 99th percentile | Removed | Extreme values distort summary statistics and visualizations |

These decisions prioritize preserving the maximum amount of analyzable data while
removing values that are clearly erroneous or would distort results.

## 3. Exploratory Data Analysis

In [ ]:
def summarize_by_group(df, group_col, agg_col="price", agg_funcs=None):
    """Compute grouped summary statistics sorted by median.

    Parameters
    ----------
    df : pd.DataFrame
        Cleaned Airbnb dataframe.
    group_col : str
        Column to group by (e.g., 'neighbourhood_group', 'room_type').
    agg_col : str
        Numeric column to aggregate. Default 'price'.
    agg_funcs : list of str or None
        Aggregation functions. Default: count, mean, median, std, min, max.

    Returns
    -------
    pd.DataFrame
        Summary table sorted by median descending.
    """
    if agg_funcs is None:
        agg_funcs = ["count", "mean", "median", "std", "min", "max"]
    summary = df.groupby(group_col)[agg_col].agg(agg_funcs)
    return summary.sort_values("median", ascending=False)

In [ ]:
print("=== Price by Borough ===\n")
summarize_by_group(df, "neighbourhood_group")

In [ ]:
print("=== Price by Room Type ===\n")
summarize_by_group(df, "room_type")

In [ ]:
print("=== Listing Counts by Borough and Room Type ===\n")
ct = pd.crosstab(df["neighbourhood_group"], df["room_type"], margins=True)
print(ct)

In [ ]:
print("=== Correlation Matrix (numeric columns) ===\n")
numeric_cols = ["price", "minimum_nights", "number_of_reviews",
                "reviews_per_month", "calculated_host_listings_count",
                "availability_365"]
corr = df[numeric_cols].corr()
print(corr.round(2))

## 4. Visualizations

In [ ]:
# Visualization 1: Price Distribution by Borough
borough_order = (df.groupby("neighbourhood_group")["price"]
                 .median().sort_values(ascending=False).index)

fig, ax = plt.subplots(figsize=(10, 6))
sns.boxplot(
    data=df,
    x="neighbourhood_group",
    y="price",
    order=borough_order,
    palette="viridis",
    fliersize=2,
    ax=ax,
)
ax.set_title("Airbnb Listing Price Distribution by NYC Borough", fontsize=14, fontweight="bold")
ax.set_xlabel("Borough", fontsize=12)
ax.set_ylabel("Price (USD per night)", fontsize=12)
plt.tight_layout()
plt.savefig("viz1_price_by_borough.png", dpi=150, bbox_inches="tight")
plt.show()

### Interpretation — Figure 1: Price Distribution by Borough

Manhattan shows the highest median nightly price, with a wide interquartile range
reflecting its diverse accommodation market (from budget shared rooms to luxury
apartments). Brooklyn follows as the second most expensive borough. Staten Island,
the Bronx, and Queens display compressed price distributions with lower medians,
consistent with their distance from Manhattan's central business districts and
tourist attractions. All boroughs show right-skewed distributions with upper
outliers, though Manhattan's outliers extend considerably higher.

In [ ]:
# Visualization 2: Room Type Composition by Borough
ct_pct = pd.crosstab(
    df["neighbourhood_group"], df["room_type"], normalize="index"
) * 100

# Sort boroughs by total listing count
borough_order_count = df["neighbourhood_group"].value_counts().index

fig, ax = plt.subplots(figsize=(10, 6))
ct_pct.loc[borough_order_count].plot(
    kind="bar",
    stacked=True,
    colormap="Set2",
    edgecolor="white",
    linewidth=0.5,
    ax=ax,
)
ax.set_title("Room Type Composition by NYC Borough", fontsize=14, fontweight="bold")
ax.set_xlabel("Borough", fontsize=12)
ax.set_ylabel("Percentage of Listings (%)", fontsize=12)
ax.legend(title="Room Type", bbox_to_anchor=(1.02, 1), loc="upper left")
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
plt.tight_layout()
plt.savefig("viz2_room_type_composition.png", dpi=150, bbox_inches="tight")
plt.show()

### Interpretation — Figure 2: Room Type Composition by Borough

Manhattan has the highest proportion of entire home/apartment listings (~60%),
reflecting its hotel-alternative market. Brooklyn shows a more balanced mix
between entire homes and private rooms. Queens, the Bronx, and Staten Island
skew more heavily toward private rooms, suggesting hosts in outer boroughs are
more likely to rent a room in their own residence rather than dedicate an entire
unit to short-term rental. Shared rooms represent a small fraction across all
boroughs.

In [ ]:
# Visualization 3: Geographic Scatter by Price
fig, ax = plt.subplots(figsize=(12, 10))
scatter = ax.scatter(
    df["longitude"],
    df["latitude"],
    c=df["price"],
    cmap="plasma",
    alpha=0.3,
    s=3,
    vmin=0,
    vmax=500,
)
cbar = plt.colorbar(scatter, ax=ax, shrink=0.7, label="Price (USD per night)")
ax.set_title(
    "Geographic Distribution of Airbnb Listings by Price in NYC",
    fontsize=14,
    fontweight="bold",
)
ax.set_xlabel("Longitude", fontsize=12)
ax.set_ylabel("Latitude", fontsize=12)
ax.set_aspect("equal")
plt.tight_layout()
plt.savefig("viz3_geographic_scatter.png", dpi=150, bbox_inches="tight")
plt.show()

### Interpretation — Figure 3: Geographic Distribution by Price

The scatter plot reveals the geographic footprint of NYC's Airbnb market. High-price
listings (bright/yellow tones) cluster heavily in lower and midtown Manhattan, as
well as along the Brooklyn waterfront near Williamsburg and DUMBO. The outer
boroughs show predominantly lower prices (darker tones) with more uniform
distributions. Listing density is highest in Manhattan and northwest Brooklyn,
visually confirming these as the primary short-term rental markets. Staten Island
has noticeably sparser coverage, consistent with its lower tourism profile.

## 5. Summary

### Key Insights

1. **Manhattan dominates pricing:** Median nightly prices in Manhattan are
   significantly higher than other boroughs, driven by proximity to tourist
   attractions and business centers.

2. **Room type varies by borough:** Entire home/apartment listings dominate
   Manhattan, while outer boroughs lean toward private rooms — reflecting
   different hosting economics and housing density.

3. **Geographic clustering:** High-price listings cluster along Manhattan's
   core and the Brooklyn waterfront, with price gradients radiating outward.

4. **Review activity correlates inversely with price:** Lower-priced listings
   tend to have more reviews, suggesting higher booking frequency at
   accessible price points.

### Patterns

- Price distributions are right-skewed across all boroughs, with Manhattan
  showing the widest spread.
- The Bronx and Staten Island represent underserved markets with fewer
  listings and lower prices.
- Seasonal review patterns (visible in `last_review` dates) suggest peak
  activity in summer months.

### Assumptions and Limitations

- **Price = $0 removed:** We assumed these are inactive or erroneous entries,
  but some may represent promotional listings.
- **99th percentile cutoff:** The upper bound removal is a common but
  arbitrary threshold — results may shift under different cutoffs.
- **Snapshot data:** This is a single point-in-time dataset (2019), so we
  cannot infer trends or seasonal dynamics.
- **Self-reported data:** Prices are set by hosts and may not reflect actual
  booking rates or market clearing prices.

### Reproducibility

This workflow follows reproducible data science practices (Danchev, 2022).
All data transformations are implemented in documented Python functions,
dependencies are pinned in `requirements.txt`, and the project is version-
controlled with Git. The dataset is included in the repository so the
notebook runs immediately after cloning without external downloads.